# Airplane Crashes — Data Cleaning

Source file: `/mnt/data/Airplane_Crashes_and_Fatalities_Since_1908.csv`

**1. Number of rows and columns**

- Rows: 5268
- Columns: 13


In [ ]:
%pip install pandas numpy

import pandas as pd
import numpy as np
df = pd.read_csv('C:\\Users\\colli\\OneDrive - Strathmore University\\AI-Coursework--2025-\\Airplane_Crashes_and_Fatalities_Since_1908.csv')
df.shape


**2. Last 75 rows of the dataset**

In [ ]:
df.tail(75)

**3. Missing values per column and recommended treatment**

In [ ]:
missing_table = pd.DataFrame({'missing_count': df.isna().sum(), 'missing_pct': (df.isna().sum()/len(df)*100).round(2)})
missing_table

In [ ]:
recommendations = ['First column — typically an identifier/date column: parse dates if needed and keep original.', "Column `Time` — missing 2219 (42.12%). Recommended: Impute with mode (most frequent) or placeholder 'Unknown'; investigate category meanings.", "Column `Location` — missing 20 (0.38%). Recommended: Impute with mode (most frequent) or placeholder 'Unknown'; investigate category meanings.", "Column `Operator` — missing 18 (0.34%). Recommended: Impute with mode (most frequent) or placeholder 'Unknown'; investigate category meanings.", 'Column `Flight #` — missing 4199 (79.71%). Recommended: Drop column or keep for reference; too many missing values to impute reliably.', "Column `Route` — missing 1706 (32.38%). Recommended: Impute with mode (most frequent) or placeholder 'Unknown'; investigate category meanings.", "Column `Type` — missing 27 (0.51%). Recommended: Impute with mode (most frequent) or placeholder 'Unknown'; investigate category meanings.", "Column `Registration` — missing 335 (6.36%). Recommended: Impute with mode (most frequent) or placeholder 'Unknown'; investigate category meanings.", "Column `cn/In` — missing 1228 (23.31%). Recommended: Impute with mode (most frequent) or placeholder 'Unknown'; investigate category meanings.", 'Column `Aboard` — missing 22 (0.42%). Recommended: Impute using median (robust to outliers) or mean if distribution is symmetric.', 'Column `Fatalities` — missing 12 (0.23%). Recommended: Impute using median (robust to outliers) or mean if distribution is symmetric.', 'Column `Ground` — missing 22 (0.42%). Recommended: Impute using median (robust to outliers) or mean if distribution is symmetric.', "Column `Summary` — missing 390 (7.4%). Recommended: Impute with mode (most frequent) or placeholder 'Unknown'; investigate category meanings."]
for r in recommendations:
    print('- ' + r)


**4. Creating `fatality_locations` dataframe**

Detected columns:
- Date: `Date`
- Location: `Location`
- Aboard: `Aboard`
- Fatalities: `Fatalities`


In [ ]:
fatality_locations = df[['Date', 'Location', 'Aboard', 'Fatalities']].copy()
fatality_locations.columns = ['Date','Location','Aboard','Fatalities']

fatality_locations['Aboard'] = pd.to_numeric(fatality_locations['Aboard'], errors='coerce')
fatality_locations['Fatalities'] = pd.to_numeric(fatality_locations['Fatalities'], errors='coerce')

fatality_locations.head()


**5. Date with the highest recorded fatalities**

In [ ]:
max_row = fatality_locations.loc[fatality_locations['Fatalities'].idxmax()]
max_row[['Date','Location','Aboard','Fatalities']]

**6. Compare number aboard vs fatalities; count crashes with zero fatalities**

In [ ]:
zero_fatal = fatality_locations[fatality_locations['Fatalities']==0]
num_zero_fatal = len(zero_fatal)
num_zero_fatal

**7. Split `Location` into `Region` and `State/Country`**

In [ ]:

def split_location(x):
    if pd.isna(x):
        return pd.Series([pd.NA, pd.NA])
    parts = [p.strip() for p in x.rsplit(',', 1)]
    if len(parts) == 2:
        return pd.Series(parts)
    else:
        return pd.Series([parts[0], pd.NA])

fatality_locations[['Region','State/Country']] = fatality_locations['Location'].apply(split_location)
fatality_locations[['Date','Location','Region','State/Country','Aboard','Fatalities']].head()


**8. Top 100 crashes ordered by Fatalities (highest → lowest)**

In [ ]:
top100 = fatality_locations.sort_values('Fatalities', ascending=False).head(100)
top100[['Date','Region','State/Country','Aboard','Fatalities']]

**9. Pie chart: Top 25 fatalities by Country/U.S. State (sum of Fatalities)**

In [ ]:

%pip install matplotlib

import matplotlib.pyplot as plt

agg = fatality_locations.copy()
agg['State/Country'] = agg['State/Country'].fillna('Unknown')
by_country = agg.groupby('State/Country')['Fatalities'].sum().sort_values(ascending=False)
top25 = by_country.head(25)

plt.figure(figsize=(10,10))
top25.plot.pie(autopct='%1.1f%%', startangle=90, ylabel='')
plt.title('Top 25 fatalities by Country / U.S. State (sum of Fatalities)')
plt.tight_layout()
plt.show()

top25
